import libaries

In [ ]:
#import libaries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
#The line magic command %matplotlib inline enables the drawing of matplotlib figures in the IPython environment

import data set 

In [ ]:
dataset = pd.read_csv("../datasets/train.csv")
dataset_test= pd.read_csv("../datasets/test.csv")

Reading the data

In [ ]:
dataset['train_test'] = 1
dataset_test['train_test'] = 0
dataset_test['Survived'] = np.NaN
all_data = pd.concat([dataset,dataset_test])

In [ ]:
all_data.columns

Data Exploration

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

seperate the data into numeric and categorical

In [ ]:
df_num = dataset[['Age','SibSp','Parch','Fare']]
df_cat = dataset[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

make plots of the numeric data:

In [ ]:
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

In [ ]:
import seaborn as sns

compare survival rates across the numeric variables, insights:

In [ ]:
sns.heatmap(df_num.corr());

In [ ]:
pd.pivot_table(dataset, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

-average age of survivors is 28, so young people tend to survive more

 -people who paid higher fare rates were more likely to survive

 -in the 3º column, if you have parents, you had a higher chance of surviving

 -if you are a child, and have siblings, you have less of a chance of surviving

In [ ]:
df_cat.Sex.value_counts()

In [ ]:
for i in df_cat.columns:
    sns.barplot(x=df_cat[i].value_counts().index, y=df_cat[i].value_counts().values).set_title(i)
    plt.show()

guess:
Survived: Most of the people died in the shipwreck

Pclass: The majority of the people had tickets to the 3rd class

Sex: There were more males than females aboard the ship, maybe double the amount

Embarked: Most boarded the ship from Southampton


 categorical variables compare them against our dependent variable, which is if people survived:

In [ ]:
print(pd.pivot_table(dataset, index = 'Survived', columns = 'Pclass',
                     values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(dataset, index = 'Survived', columns = 'Sex', 
                     values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(dataset, index = 'Survived', columns = 'Embarked', 
                     values = 'Ticket' ,aggfunc ='count'))

-Pclass: lot more people survived from the First class than the Second or the Third class

-Sex: Most of the women survived, and the majority of the male died 

-Embarked: This doesn’t seem much relevant

"""Feature Engineering"""

-ticket and cabin data don’t really make sense

-this might hinder the performance of the model

 -to simplify this data with feature engineering

-first split into individual cabins and see whether someone owned more than a single cabin

In [ ]:
df_cat.Cabin
dataset['cabin_multiple'] = dataset.Cabin.apply(lambda x: 0 if pd.isna(x) 
                                                    else len(x.split(' ')))
dataset['cabin_multiple'].value_counts()

- It looks like the vmajority did not have individual cabins, a only a few people owned more than one cabins

- Now  see whether the survival rates depend on this

In [ ]:
pd.pivot_table(dataset, index = 'Survived', columns = 'cabin_multiple',
               values = 'Ticket' ,aggfunc ='count')

-look at the actual letter of the cabin they were in

-Cabins with the same letter are roughly in the same locations

-look into that:

In [ ]:
dataset['cabin_adv'] = dataset.Cabin.apply(lambda x: str(x)[0])
#comparing survival rates by cabin
print(dataset.cabin_adv.value_counts())
pd.pivot_table(dataset,index='Survived',columns='cabin_adv', 
                        values = 'Name', aggfunc='count')

divide the tickets into numeric and non-numeric for efficient usage:

In [ ]:
dataset['numeric_ticket'] = dataset.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
dataset['ticket_letters'] = dataset.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1])
                                            .replace('.','').replace('/','')
                                            .lower() if len(x.split(' ')[:-1]) >0 else 0)

look at is the title of individual passengers

In [ ]:
dataset.Name.head(50)
dataset['name_title'] = dataset.Name.apply(lambda x: x.split(',')[1]
                                                        .split('.')[0].strip())
dataset['name_title'].value_counts()